In [ ]:
# Análise Genérica de Dados de Criminalidade
# ------------------------------------------

# Importações
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

from openpyxl import load_workbook
from IPython.display import display

# Configurações de exibição
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
